In [3]:
import numpy as np
import sys
import tensorflow as tf
from glob import glob
import os
sys.path.append('../')
sys.path.append('./../src/')
import ScalableLib.classifier.Multiband as multiband

import classifier.Multiband as multiband
import base.plot as plot

# To reset cuda
# sudo rmmod nvidia_uvm
# sudo modprobe nvidia_uvm
# To see if the system recognises the GPU
device = 1
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

survey = 'Gaia'
path = './Results/Fold_*'
folds = glob(path)
folds.sort()

if not os.path.exists('./Results/'):
    os.mkdir('./Results/')

label_order = ['CEP', 'T2CEP', 'MIRA_SR', 'RRAB', 'RRC', 'DSCT_SXPHE']

# Containers to store the results
reports_folds = []
cm_folds = []
regressions = []

ModuleNotFoundError: No module named 'base'

In [21]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold + '/'
    # base_dir = fold
    # Get the last run
    path_runs_folder = os.path.join(base_dir, 'Models', '*')
    path_runs = glob(path_runs_folder)
    path_runs.sort()
    path_run = path_runs[-1]

    # path_preprocess = fold.replace('/03_TrainModels/', '/02_CreateRecords/')
    path_preprocess = fold.replace('Results', 'Folds')
    test_files = os.path.join(path_preprocess, 'test/*.tfrecord')

    new = multiband.Network()
    settings_path = os.path.join(path_run, 'all_settings.json')
    # new.batch_size = 512
    fold_name = fold.split('/')[-1]
    write_path = os.path.join('./Results', fold_name + '_Results.dat')

    # Get weights path
    run = path_run.split('/')[-1]
    weights_path = os.path.join(fold, 'Models', run)
    new.run_test(settings_path, test_files, weights_path, df_paths=write_path)

    break

./Folds/Fold_1/test/*.tfrecord


2023-07-17 18:47:42.999019: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101


              precision    recall  f1-score   support

         CEP       0.00      0.00      0.00      1166
  DSCT_SXPHE       0.17      0.40      0.24       680
     MIRA_SR       0.31      0.85      0.45      1006
        RRAB       0.19      0.01      0.01       728
         RRC       0.23      0.12      0.15      1073
         RRD       0.00      0.00      0.00        92
       T2CEP       0.00      0.00      0.00       236

    accuracy                           0.25      4981
   macro avg       0.13      0.20      0.12      4981
weighted avg       0.16      0.25      0.16      4981

{'R2': {'T_eff': -3.7002729489807695, 'Radius': -0.012305016448439954}, 'RMSE': {'T_eff': 2514.4458, 'Radius': 24.300797}}


/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels w

In [3]:
new.run_test_test(settings_path, test_files, weights_path, df_paths=write_path)

2023-07-17 18:42:43.158848: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 18:42:43.159995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-17 18:42:43.160554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-17 18:42:43.160981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

./Fold/Fold_1/test/*.tfrecord


In [14]:
import base.Parser as Parser
def initialize_dataset_test( filename_test):
    loader = Parser.Parser(physical_parameters=new.physical_params,
                           n_bands=new.n_bands,
                           num_classes=new.num_classes,
                           w=new.w,
                           batch_size=new.batch_size,
                           num_threads=new.num_cores,
                           buffer_size=new.buffer_size,
                           mode=new.mode,
                           )
    print(filename_test)
    new.dataset_test = loader.get_dataset(filename=filename_test,
                                           epochs=1,
                                           shuffle=False
                                           )

In [19]:
initialize_dataset_test([test_files])

['./Fold/Fold_1/test/*.tfrecord']


In [20]:
for batch in new.dataset_test:
    break
batch

NotFoundError: ./Fold/Fold_1/test/*.tfrecord; No such file or directory [Op:IteratorGetNext]

In [17]:
path_parameters=settings_path
path_records_test=test_files
path_weights=weights_path

In [19]:
import pickle
new.load_setup(path_parameters)
# Load scalers
if 'regression' in new.mode:
    # Load the scalers
    with open(new.path_scalers, 'rb') as file:
        new.scalers = pickle.load(file)
# Initialize dataset
new.__initialize_dataset_test(path_records_test)

AttributeError: 'Network' object has no attribute '__initialize_dataset_test'